# 1. 웹 스크래핑

이번 강의에서는 웹 스크래핑에 대해 배웠다.

여기선 페이커의 플레이 정보를 OPGG 웹 사이트에서 불러올 것이다.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=Hide%20on%20bush'

# 페이지의 html 코드를 faker_html로 저장합니다.
faker_html = requests.get(faker_opgg_url).text

# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
faker_soup = BeautifulSoup(faker_html, "html.parser")

- `BeautifulSoup`을 이용한 웹 스크래핑은 이전에 [나도코딩 웹 스크래핑](https://romg2.github.io/categories/%EB%82%98%EB%8F%84%EC%BD%94%EB%94%A9)에서 공부하여 정리한 경험이 있다.

In [2]:
# faker_soup

- HTML 파일을 확인 가능하나 매우 기니까 원한다면 주석 해제하여 구조를 살펴보자.

In [3]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_recent_champions = []
faker_recent_kills = []
faker_recent_deaths = []
faker_recent_assists = []
faker_recent_results = []

In [4]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
faker_recent_games_html = faker_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 한 페이지에 최초로 로딩된 최근 게임 수 출력
recent_game_len = len(faker_recent_games_html)
print(recent_game_len)

20


- OPGG에선 한 페이지에 최근 20게임이 로드된다.


- `select()`등 워낙 다양한 메소드가 많으니 직접 해보면서 찾아보는 것이 중요하다.

In [5]:
type(faker_recent_games_html)

list

- `select()` 사용하면 리스트 형태로 저장된다.

In [6]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(recent_game_len):
    faker_recent_champions.append(''.join(list(faker_recent_games_html[i].select('div.ChampionName')[0].
                                               stripped_strings)))
    faker_recent_kills.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Kill')[0].
                                               stripped_strings)[0])
    faker_recent_deaths.append(list(faker_recent_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    faker_recent_assists.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Assist')[0].
                                               stripped_strings)[0])
    faker_recent_results.append(list(faker_recent_games_html[i].select('div.GameStats div.GameResult')[0].
                                               stripped_strings)[0])
#     아래 코드는 data-game-result라는 attr 가져오는 방법 다만 다시하기도 lose라 되있어서 사용 x
#     faker_recent_results.append(faker_recent_games_html[i].select_one('div.GameItem')['data-game-result'])

- 각 태그들은 본인이 직접 F12(개발자 모드)로 확인하면서 지정하면 된다.

In [7]:
# DataFrame으로 변환 후 출력
faker_recent_df = pd.DataFrame([faker_recent_champions,
                                faker_recent_results,
                                faker_recent_kills,
                                faker_recent_deaths,
                                faker_recent_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_recent_df

,champion,result,kills,deaths,assists
0,Orianna,Victory,10,3,18
1,Lee Sin,Victory,8,4,14
2,Anivia,Defeat,7,5,10
3,LeBlanc,Victory,4,2,4
4,Blitzcrank,Victory,4,1,17
5,Orianna,Defeat,3,3,0
6,Orianna,Defeat,2,7,3
7,Tristana,Victory,9,8,9
8,Vayne,Victory,8,5,10
9,Aphelios,Victory,5,7,2


- 페이커의 최근 플레이한 정보를 위와 같이 OPGG 사이트에서 잘 불러왔다.


- 아래 코드들은 출력 형태에 대해 파악해보려고 작성한 것이다.

In [8]:
faker_recent_games_html[0].select('div.ChampionName')

[<div class="ChampionName">
 <a href="/champion/orianna/statistics" target="_blank">Orianna</a>
 </div>]

In [9]:
faker_recent_games_html[0].find("div", attrs={"class":"ChampionName"})

<div class="ChampionName">
<a href="/champion/orianna/statistics" target="_blank">Orianna</a>
</div>

- `find()` 메소드를 사용해도 된다.


- `select()`와는 다르게 리스트 형태가 아닌 기존 값?을 그대로 출력한다.

In [10]:
faker_recent_games_html[0].select('div.ChampionName')[0].get_text()

'\nOrianna\n'

- `select()`의 리스트에서 첫 번째 값을 `get_text()`하면 위와 같은 형태이다.

In [11]:
list(faker_recent_games_html[0].select('div.ChampionName')[0].strings)

['\n', 'Orianna', '\n']

- `strings`를 사용하면 다음과 같이 분리되어 리스트로 나타난다.

In [12]:
list(faker_recent_games_html[0].select('div.ChampionName')[0].stripped_strings)

['Orianna']

- `stripped_strings`을 사용하면 불필요한 문자는 지워지고 리스트로 나타난다.